# **Deforum Google Colab**
Deforum for Google Colab. Generate animations with diffusion models. Features include: text-to-video, image-to-video, video-to-video, and keyframe camera motions. Supports SD1.5, SDXL, and SD3. Built on top of diffusers. MIT license. Notebook by [Huemin](https://www.huemin.art)

In [1]:
#@markdown **SETUP ENVIRONMENT**
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

models_path = "/content/models"
output_path = "/content/output"

# Mount Google Drive
mount_google_drive = True
force_remount = False

if mount_google_drive:
    from google.colab import drive
    try:
        drive_path = "/content/drive"
        drive.mount(drive_path,force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models"
        output_path_gdrive = "/content/drive/MyDrive/AI/outputs"
        models_path = models_path_gdrive
        output_path = output_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")

import os
os.makedirs(models_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

# install stuff
!git clone https://github.com/huemin-art/deforum-google-colab.git
!pip install -q diffusers transformers accelerate

Tesla T4, 15360 MiB, 15101 MiB
Local Path Variables:

Mounted at /content/drive
models_path: /content/drive/MyDrive/AI/models
output_path: /content/drive/MyDrive/AI/outputs
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.7 MB/s eta 0:00:00
Cloning into 'deforum-google-colab'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), done.
Resolving deltas: 100% (1/1), done.


In [14]:
from types import SimpleNamespace
from pydantic import BaseModel
import torch
from diffusers import AutoPipelineForText2Image, AutoPipelineForImage2Image

MODEL_NAME = "stabilityai/stable-diffusion-xl-base-1.0"
DEVICE = "cuda"
VARIANT = "fp16"
SAFETENSORS = True
TORCH_DTYPE = torch.float16

txt2img = AutoPipelineForText2Image.from_pretrained(
    MODEL_NAME,
    torch_dtype = TORCH_DTYPE,
    variant = VARIANT,
    use_safetensors = SAFETENSORS
).to(DEVICE)

img2img = AutoPipelineForImage2Image.from_pipe(txt2img)

class DeforumArgs(BaseModel):
    prompt: str = "cat sushi" #@param {type:"string"}
    negative_prompt: str = "" #@param {type:"string"}


def deforum_pipeline(args):

    img = txt2img(
        prompt = args.prompt,
        negative_prompt = args.prompt
    )

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
import pydantic

In [12]:
img = txt2img(prompt="cat sushi")

  0%|          | 0/50 [00:00<?, ?it/s]